In [3]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
from xgboost import XGBClassifier
import seaborn as sns 
from sklearn.preprocessing import OrdinalEncoder

In [9]:
X_train = pd.read_csv('./X_train.csv')
X_test = pd.read_csv('./X_test.csv')
train_data = pd.read_csv('./train.csv')

In [10]:
order = ['Insufficient_Weight', 'Normal_Weight',  'Overweight_Level_I',
        'Overweight_Level_II','Obesity_Type_I',
        'Obesity_Type_II', 'Obesity_Type_III']
y_ord = OrdinalEncoder(categories=[order])
y_encoded = y_ord.fit_transform(train_data[['NObeyesdad']])
y_encoded = pd.Series(y_encoded.reshape(-1),name = 'NObeyesdad')
y_encoded

0        3.0
1        1.0
2        0.0
3        6.0
4        3.0
        ... 
20753    5.0
20754    0.0
20755    5.0
20756    3.0
20757    5.0
Name: NObeyesdad, Length: 20758, dtype: float64

In [28]:
xgb_clf = XGBClassifier(objective='multi:softmax', 
                            num_class=7, 
                            tree_methord = 'gpu_hist',
                            eval_metric=['merror','mlogloss'],
                            device = 'cuda'
                            )

In [32]:
params = {'learning_rate': [0.06,0.1], 'max_depth': [3,4,5,6,7], 'n_estimators': [100,400,800]
         # 'gamma': [0],
         #  'subsample' : [0.9],
         #  'alpha': [0.005]
         }


In [18]:
from datetime import datetime
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))


In [15]:
X_train.shape,y_encoded.shape,X_test.shape

((20758, 19), (20758,), (13840, 19))

In [33]:
from sklearn.model_selection import GridSearchCV
start_time = timer(None)
grid = GridSearchCV(xgb_clf,param_grid= params,cv = 5,n_jobs=-1)
grid.fit(X_train,y_encoded)
timer(start_time)
print('\n Best estimator:')
print(grid.best_estimator_)
print('\n Best score:')
print(grid.best_score_ * 2 - 1)
print('\n Best parameters:')
print(grid.best_params_)

C:\Users\abhir\AppData\Local\Programs\Python\Python311\Lib\site-packages\xgboost\core.py:160: UserWarning: [12:49:48] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0b3782d1791676daf-1\xgboost\xgboost-ci-windows\src\learner.cc:742: 
Parameters: { "tree_methord" } are not used.

  warnings.warn(smsg, UserWarning)



 Time taken: 0 hours 22 minutes and 1.09 seconds.

 Best estimator:
XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device='cuda', early_stopping_rounds=None,
              enable_categorical=False, eval_metric=['merror', 'mlogloss'],
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=0.06, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=100,
              n_jobs=None, num_class=7, num_parallel_tree=None, ...)

 Best score:
0.7933318062040438

 Best parameters:
{'learning_rate': 0.06, 'max_depth': 6, 'n_estimators': 100}


In [49]:
pred = grid.best_estimator_.predict(X_test)
print(len(pred))
y_test = []

13840


In [50]:
for i in range(len(pred)):
    y_test.append(order[pred[i]])
y_test[:5],pred[:5],order

(['Obesity_Type_II',
  'Overweight_Level_I',
  'Obesity_Type_III',
  'Obesity_Type_I',
  'Obesity_Type_III'],
 array([5, 2, 6, 4, 6]),
 ['Insufficient_Weight',
  'Normal_Weight',
  'Overweight_Level_I',
  'Overweight_Level_II',
  'Obesity_Type_I',
  'Obesity_Type_II',
  'Obesity_Type_III'])

In [51]:
len(y_test)

13840

In [52]:
test_data = pd.read_csv('./test.csv')
test_data.shape,len(y_test)

((13840, 17), 13840)

In [53]:
y_test = pd.DataFrame({'id':test_data.id,'NObeyesdad':y_test})
y_test.head()

,id,NObeyesdad
0,20758,Obesity_Type_II
1,20759,Overweight_Level_I
2,20760,Obesity_Type_III
3,20761,Obesity_Type_I
4,20762,Obesity_Type_III


In [54]:
y_test.to_csv('submission1.csv',index=False)

In [55]:
X_test.shape

(13840, 19)

In [57]:
y_encoded.to_csv('y_test.csv',index=False)